# COMMOT test

In [15]:
import anndata as ad
import anndata
import os,re,sys
import pandas as pd
import numpy as np
import scanpy as sc
import commot as ct
import matplotlib.pyplot as plt
from plotnine import *

In [16]:
#adata_path = '/DATA/User/panhailin/project/mouse_embryo/new/E16.5_E1S3/02.limb1/limb1_cluster_deg_anno.h5ad'
adata_path = "commot_adata_limb1_vf.h5ad"
adata = ad.read(adata_path)

In [3]:
adata.X = adata.layers['raw'].copy()

In [4]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

In [72]:
df_cellchat = ct.pp.ligand_receptor_database(species='mouse', signaling_type='Secreted Signaling', database='CellChat')
#df_cellchat.to_csv("cellchat_csv.csv")

# These are immune related pathways, but the expression is too low in limb data
#df_cellchat = df_cellchat[df_cellchat['2'].isin(['COMPLEMENT','CCL','IL2','IL4','IL6','IL12','IL10','IL1','IL17','IL16','ANNEXIN', 'SAA'])]
#df_cellchat = df_cellchat[df_cellchat['2'].isin(['CCL'])]

In [73]:
df_cellchat

,0,1,2,3
0,Tgfb1,Tgfbr1_Tgfbr2,TGFb,Secreted Signaling
1,Tgfb2,Tgfbr1_Tgfbr2,TGFb,Secreted Signaling
2,Tgfb3,Tgfbr1_Tgfbr2,TGFb,Secreted Signaling
3,Tgfb1,Acvr1b_Tgfbr2,TGFb,Secreted Signaling
4,Tgfb1,Acvr1c_Tgfbr2,TGFb,Secreted Signaling
...,...,...,...,...
1204,Uts2b,Uts2r,UROTENSIN,Secreted Signaling
1205,Uts2b,Sstr5,UROTENSIN,Secreted Signaling
1206,Bag6,Ncr3-ps,BAG,Secreted Signaling
1207,Lgals9,Havcr2,GALECTIN,Secreted Signaling


In [ ]:
select_df = pd.DataFrame({'0':['Bmp3','Bmp3','Bmp3'],'1':['Bmpr1a','Bmpr1b','Bmpr2'],'2':['BMP','BMP','BMP'],'3':['Secreted Signaling','Secreted Signaling','Secreted Signaling']})
select_df

,0,1,2,3
0,Bmp3,Bmpr1a,BMP,Secreted Signaling
1,Bmp3,Bmpr1b,BMP,Secreted Signaling
2,Bmp3,Bmpr2,BMP,Secreted Signaling


In [214]:
adata_subset = adata[adata.obs['group_anno'].isin(['2 Immature_chondrocyte', '3 Mesenchymal_cell'])].copy()

In [215]:
ct.tl.spatial_communication(
    adata_subset,
    database_name='user_database', 
    df_ligrec=select_df, 
    dis_thr=2000, 
    heteromeric=True, pathway_sum=True
)

In [216]:
for i in range(len(select_df)):
    ct.tl.communication_direction(
        adata_subset, 
        database_name='user_database',
        lr_pair=(select_df['0'][i],select_df['1'][i]),
        k=10,
    )
ct.tl.communication_direction(
    adata_subset, 
    database_name='user_database',
    pathway_name="BMP",
    k=10,
)
adata_subset

AnnData object with n_obs × n_vars = 1700 × 11794
    obs: 'area', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'louvain', 'group_anno', 'highlight'
    uns: '__type', 'commot-user_database-info', 'dendrogram_louvain', 'group_anno_colors', 'log1p', 'louvain', 'louvain_colors', 'neighbors', 'pp', 'spatial'
    obsm: 'X_pca', 'X_spatial', 'X_umap', 'bbox', 'commot-user_database-sum-receiver', 'commot-user_database-sum-sender', 'commot_receiver_vf-user_database-Angptl4-Cdh11', 'commot_receiver_vf-user_database-Angptl4-Sdc1', 'commot_receiver_vf-user_database-Angptl4-Sdc3', 'commot_receiver_vf-user_database-Mdk-Itga6_Itgb1', 'commot_receiver_vf-user_database-Postn-Itgav_Itgb5', 'commot_receiver_vf-user_database-Pros1-Axl', 'commot_sender_vf-user_database-Angptl4-Cdh11', 'commot_sender_vf-user_database-Angptl4-Sdc1', 'commot_sender_vf-user_database-Angptl4-Sdc3', 'commot_sender_vf-user_database-Mdk-Itga6_Itgb1', 'commot_sender_vf-user_database-Postn

In [217]:
del adata_subset.uns['group_anno_colors']
del adata_subset.uns['louvain_colors']
adata_subset.write("tmptmp.h5ad")

In [ ]:
import anndata as ad
import anndata
import os,re,sys
import pandas as pd
import numpy as np
import scanpy as sc

import dynamo as dyn

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
plt.rcParams['image.interpolation'] = 'none'
%config InlineBackend.figure_format='retina'


import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


/home/jingzh/.conda/envs/spateo_dev/lib/python3.8/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.3 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.3, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [ ]:
adata_subset.obsm['velocity_spatial'] = adata_subset.obsm['commot_sender_vf-user_database-BMP'].copy()

dyn.pl.streamline_plot(adata_subset, color="group_anno",
                       highlights="2 Immature_chondrocyte",
                       basis="spatial", pointsize=0.1,show_legend="upper left",s_kwargs_dict={"alpha":0.5}, #color_key=[(92/255,58/255,124/255),(135/255,108/255,217/255)],
                      save_show_or_return='save',
                     save_kwargs={"ext":'png',"dpi":1000,"prefix":"limb_subset23_Bmp3_sender"},
                      )

In [ ]:
adata_subset.obsm['velocity_spatial'] = adata_subset.obsm['commot_receiver_vf-user_database-BMP'].copy()
dyn.pl.streamline_plot(adata_subset, color="group_anno",
                       highlights="2 Immature_chondrocyte",
                       basis="spatial", pointsize=0.1,show_legend="upper left",s_kwargs_dict={"alpha":0.5},
                        save_show_or_return='save',
                     save_kwargs={"ext":'png',"dpi":1000,"prefix":"limb_subset23_Bmpr1a_Bmpr1b_Bmpr2_receiver"},
                      )